In [42]:
class TransactionQueue:
    def __init__(self):
        self.first_transaction = None
        self.last_transaction = None
        self.num_transactions = 0

    def add_transaction(self, transaction):
        if self.first_transaction is None:
            self.first_transaction = transaction
        else:
            self.last_transaction.next = transaction
        self.last_transaction = transaction
        self.num_transactions += 1
    
    def remove_transaction(self):
        if self.first_transaction is None:
            raise EmptyQueueException()
        removed = self.first_transaction
        self.first_transaction = self.first_transaction.next
        if self.first_transaction is None:
            self.last_transaction = None
        self.num_transactions -= 1
        return removed

    def size(self):
        return self.num_transactions

In [43]:
# transaction_block.py

class MerkleTree:
    def __init__(self):
        self.rootnode = None
        self.numdocs = 0

    class TreeNode:
        def __init__(self):
            self.parent = None
            self.left = None
            self.right = None
            self.val = None

    def node_init(self, node, left, right, parent, val):
        node.left = left
        node.right = right
        node.parent = parent
        node.val = val

    def get_str(self, tr):
        if tr is None:
            return "None_Transaction"
            
        # Safely handle all possible None cases
        coin_id = tr.coin_id if hasattr(tr, 'coin_id') else "None"
        source_uid = tr.source.uid if hasattr(tr, 'source') and tr.source else "Genesis"
        dest_uid = tr.destination.uid if hasattr(tr, 'destination') and tr.destination else "Genesis"
        block_dgst = tr.coinsrc_block.dgst if hasattr(tr, 'coinsrc_block') and tr.coinsrc_block else "Genesis"
        
        return f"{coin_id}#{source_uid}#{dest_uid}#{block_dgst}"

    def build(self, trarray):
        if not trarray:
            return ""
            
        # Build leaf nodes
        nodes = []
        for tr in trarray:
            node = self.TreeNode()
            self.node_init(node, None, None, None, self.get_str(tr))
            nodes.append(node)
        
        # Build tree levels
        while len(nodes) > 1:
            new_level = []
            for i in range(0, len(nodes), 2):
                left = nodes[i]
                right = nodes[i+1] if i+1 < len(nodes) else None
                
                parent = self.TreeNode()
                left_val = left.val if left else ""
                right_val = right.val if right else ""
                parent_val = f"{left_val}#{right_val}"  # In reality, hash this with CRF
                
                self.node_init(parent, left, right, None, parent_val)
                if left:
                    left.parent = parent
                if right:
                    right.parent = parent
                new_level.append(parent)
            nodes = new_level
        
        if nodes:
            self.rootnode = nodes[0]
            return self.rootnode.val
        return ""

class TransactionBlock:
    def __init__(self, transactions):
        self.trarray = transactions.copy()
        self.previous = None
        self.tree = MerkleTree()
        self.trsummary = self.tree.build(self.trarray)
        self.nonce = None
        self.dgst = None

    def check_transaction(self, t):
        if t is None:
            return False
            
        if not hasattr(t, 'coinsrc_block') or t.coinsrc_block is None:
            return True
        
        # Check if coin exists in source block
        found_in_source = False
        for t_prime in t.coinsrc_block.trarray:
            if (t_prime and 
                hasattr(t_prime, 'coin_id') and 
                hasattr(t_prime, 'destination') and
                t_prime.destination and
                t_prime.coin_id == t.coin_id and 
                t_prime.destination.uid == t.source.uid):
                found_in_source = True
                break
                
        if not found_in_source:
            return False
        
        # Check for double spending
        current = self
        while current != t.coinsrc_block:
            for tx in current.trarray:
                if (tx and 
                    hasattr(tx, 'coin_id') and 
                    tx.coin_id == t.coin_id):
                    return False
            if current.previous is None:
                break
            current = current.previous
        return True

# Test Code
if __name__ == "__main__":
    class Member:
        def __init__(self, uid):
            self.uid = uid

    class Transaction:
        def __init__(self):
            self.coin_id = None
            self.source = None
            self.destination = None
            self.coinsrc_block = None

    print("Testing TransactionBlock...")
    
    # Create test members
    alice = Member("101")
    bob = Member("102")
    
    # Create test transactions
    tx1 = Transaction()
    tx1.coin_id = "100001"
    tx1.source = None  # Moderator transaction
    tx1.destination = alice
    
    tx2 = Transaction()
    tx2.coin_id = "100002"
    tx2.source = alice
    tx2.destination = bob
    
    # Create a block
    genesis_block = TransactionBlock([tx1])
    print(f"Genesis block created with trsummary: {genesis_block.trsummary}")
    
    # Test transaction validation
    print(f"Checking tx2 (should be False): {genesis_block.check_transaction(tx2)}")
    
    # Set up tx2's coinsrc_block
    tx2.coinsrc_block = genesis_block
    second_block = TransactionBlock([tx2])
    print(f"Second block created with trsummary: {second_block.trsummary}")
    print(f"Checking tx2 again (should be True): {second_block.check_transaction(tx2)}")

Testing TransactionBlock...
Genesis block created with trsummary: 100001#Genesis#101#Genesis
Checking tx2 (should be False): True
Second block created with trsummary: 100002#101#102#None
Checking tx2 again (should be True): False


In [44]:
# blockchain_honest.py

# First, define the CRF class directly in the file
class CRF:
    def __init__(self, size):
        self.outputsize = size
        assert self.outputsize <= 64

    def fn(self, s):
        import hashlib
        # Simple SHA-256 implementation for demonstration
        # Replace with your actual CRF implementation if different
        hash_object = hashlib.sha256(s.encode())
        hex_dig = hash_object.hexdigest()
        return hex_dig[:self.outputsize]

class BlockChainHonest:
    def __init__(self):
        self.tr_count = 4  # Default value, can be changed
        self.start_string = "DSCoin"
        self.last_block = None

    def insert_block_honest(self, new_block):
        crf = CRF(64)
        prefix = "0000"
        nonce = 1000000001
        
        prev_digest = self.start_string if self.last_block is None else self.last_block.dgst
        
        while True:
            digest = crf.fn(f"{prev_digest}#{new_block.trsummary}#{nonce}")
            if digest.startswith(prefix):
                new_block.nonce = str(nonce)
                new_block.dgst = digest
                break
            nonce += 1
        
        new_block.previous = self.last_block
        self.last_block = new_block

# Test code
if __name__ == "__main__":
    # Mock TransactionBlock class for testing
    class TransactionBlock:
        def __init__(self):
            self.trsummary = "test_summary"
            self.nonce = None
            self.dgst = None
            self.previous = None
    
    print("Testing BlockChainHonest...")
    blockchain = BlockChainHonest()
    block = TransactionBlock()
    
    print("Inserting first block...")
    blockchain.insert_block_honest(block)
    print(f"Block nonce: {block.nonce}")
    print(f"Block digest: {block.dgst}")
    
    print("Inserting second block...")
    block2 = TransactionBlock()
    blockchain.insert_block_honest(block2)
    print(f"Block2 nonce: {block2.nonce}")
    print(f"Block2 digest: {block2.dgst}")
    print(f"Block2 points to previous block: {block2.previous == block}")

Testing BlockChainHonest...
Inserting first block...
Block nonce: 1000041548
Block digest: 0000e4ffc3fe45977eb13ce4a149a89c6c4b5b8b6e38f3172f0395ad9a74b513
Inserting second block...
Block2 nonce: 1000196282
Block2 digest: 00001ec676a8b9f1ebc8306861198639c1b24b384cd57e67fd22b0ba036c614c
Block2 points to previous block: True


In [45]:
# blockchain_malicious.py

# Include CRF implementation directly in the file
class CRF:
    def __init__(self, size):
        self.outputsize = size
        assert self.outputsize <= 64

    def fn(self, s):
        import hashlib
        hash_object = hashlib.sha256(s.encode())
        return hash_object.hexdigest()[:self.outputsize]

# Include MerkleTree implementation directly in the file
class MerkleTree:
    def __init__(self):
        self.rootnode = None
        self.numdocs = 0

    class TreeNode:
        def __init__(self):
            self.parent = None
            self.left = None
            self.right = None
            self.val = None

    def build(self, trarray):
        # Simplified implementation for demo
        if not trarray:
            return ""
        return "mock_merkle_root"  # Replace with actual implementation

class BlockChainMalicious:
    def __init__(self):
        self.tr_count = 4  # Default value, can be changed
        self.start_string = "DSCoin"
        self.last_blocks_list = [None] * 100  # Assuming max 100 last blocks

    def check_transaction_block(self, tb):
        crf = CRF(64)
        prev_digest = self.start_string if tb.previous is None else tb.previous.dgst
        computed_digest = crf.fn(f"{prev_digest}#{tb.trsummary}#{tb.nonce}")
        
        if computed_digest != tb.dgst:
            return False
        if not tb.dgst.startswith("0000"):
            return False
        
        # Check Merkle tree (simplified for demo)
        mt = MerkleTree()
        if mt.build(tb.trarray) != tb.trsummary:
            return False
        
        # Transaction validation would go here
        return True

    def find_longest_valid_chain(self):
        longest_chain_end = None
        max_length = 0
        
        for last_block in self.last_blocks_list:
            if last_block is None:
                continue
            
            current = last_block
            current_length = 0
            valid_end = None
            
            while current is not None:
                if self.check_transaction_block(current):
                    current_length += 1
                    valid_end = current
                    current = current.previous
                else:
                    if current_length > max_length:
                        max_length = current_length
                        longest_chain_end = valid_end
                    current_length = 0
                    current = current.previous
            
            if current_length > max_length:
                max_length = current_length
                longest_chain_end = valid_end
        
        return longest_chain_end

    def insert_block_malicious(self, new_block):
        last_valid = self.find_longest_valid_chain()
        
        crf = CRF(64)
        prefix = "0000"
        nonce = 1000000001
        
        prev_digest = self.start_string if last_valid is None else last_valid.dgst
        
        while True:
            digest = crf.fn(f"{prev_digest}#{new_block.trsummary}#{nonce}")
            if digest.startswith(prefix):
                new_block.nonce = str(nonce)
                new_block.dgst = digest
                break
            nonce += 1
        
        new_block.previous = last_valid
        
        # Update last_blocks_list
        for i in range(len(self.last_blocks_list)):
            if self.last_blocks_list[i] is None:
                self.last_blocks_list[i] = new_block
                break

# Test code
if __name__ == "__main__":
    print("Testing BlockChainMalicious...")
    
    # Mock TransactionBlock class
    class TransactionBlock:
        def __init__(self):
            self.trarray = []
            self.trsummary = "test_summary"
            self.nonce = None
            self.dgst = None
            self.previous = None
    
    # Create blockchain
    blockchain = BlockChainMalicious()
    
    # Create and insert first block
    block1 = TransactionBlock()
    blockchain.insert_block_malicious(block1)
    print(f"Block1 nonce: {block1.nonce}")
    print(f"Block1 digest: {block1.dgst}")
    
    # Create and insert second block
    block2 = TransactionBlock()
    blockchain.insert_block_malicious(block2)
    print(f"Block2 nonce: {block2.nonce}")
    print(f"Block2 digest: {block2.dgst}")
    print(f"Block2 points to: {'Genesis' if block2.previous is None else 'Block1'}")

Testing BlockChainMalicious...
Block1 nonce: 1000041548
Block1 digest: 0000e4ffc3fe45977eb13ce4a149a89c6c4b5b8b6e38f3172f0395ad9a74b513
Block2 nonce: 1000041548
Block2 digest: 0000e4ffc3fe45977eb13ce4a149a89c6c4b5b8b6e38f3172f0395ad9a74b513
Block2 points to: Genesis


In [46]:
# members.py

# First, define the Pair class directly in the file
class Pair:
    def __init__(self, first, second):
        self.first = first
        self.second = second
    
    def get_first(self):
        return self.first
    
    def get_second(self):
        return self.second

# Define a simple Transaction class
class Transaction:
    def __init__(self):
        self.coin_id = None
        self.source = None
        self.destination = None
        self.coinsrc_block = None
        self.next = None  # For queue implementation

# Define a simple TransactionBlock class
class TransactionBlock:
    def __init__(self, transactions):
        self.trarray = transactions
        self.previous = None
        self.dgst = None
        self.trsummary = "mock_trsummary"
        self.nonce = None

# Define custom exceptions
class MissingTransactionException(Exception):
    pass

class EmptyQueueException(Exception):
    pass

# Main Members class implementation
class Members:
    def __init__(self, uid):
        self.uid = uid
        self.mycoins = []  # List of Pair(coin_id, TransactionBlock)
        self.in_process_trans = [None] * 100  # Assuming max 100 in-process transactions

    def initiate_coinsend(self, dest_uid, ds_obj):
        if not self.mycoins:
            return
        
        coin_pair = self.mycoins.pop(0)
        t = Transaction()
        t.coin_id = coin_pair.first
        t.source = self
        t.destination = self.find_member(dest_uid, ds_obj.memberlist)
        t.coinsrc_block = coin_pair.second
        
        # Add to in_process and pending transactions
        self.add_to_in_process(t)
        ds_obj.pending_transactions.add_transaction(t)

    def finalize_coinsend(self, tobj, ds_obj):
        # Find transaction in blockchain
        t_block = self.find_transaction_block(tobj, ds_obj.b_chain.last_block)
        if t_block is None:
            raise MissingTransactionException()
        
        # Build sibling-coupled path (simplified for demo)
        path = [Pair("path1", "path2")]  # Mock path
        
        # Build block proofs (simplified for demo)
        proofs = [Pair("proof1", "proof2")]  # Mock proofs
        
        # Update coin ownership
        self.remove_from_in_process(tobj)
        tobj.destination.mycoins.append(Pair(tobj.coin_id, t_block))
        
        return Pair(path, proofs)

    def mine_coin(self, ds_obj):
        transactions = []
        try:
            for _ in range(ds_obj.b_chain.tr_count - 1):
                transactions.append(ds_obj.pending_transactions.remove_transaction())
        except EmptyQueueException:
            return

        # Add miner reward transaction
        new_coin_id = str(int(ds_obj.latest_coin_id) + 1)
        reward = Transaction()
        reward.coin_id = new_coin_id
        reward.source = None
        reward.destination = self
        reward.coinsrc_block = None
        transactions.append(reward)
        
        # Create and insert block
        new_block = TransactionBlock(transactions)
        if hasattr(ds_obj.b_chain, 'insert_block_honest'):
            ds_obj.b_chain.insert_block_honest(new_block)
        else:
            ds_obj.b_chain.insert_block_malicious(new_block)
        
        # Update miner's coins
        self.mycoins.append(Pair(new_coin_id, new_block))
        ds_obj.latest_coin_id = new_coin_id

    # Helper methods
    def find_member(self, uid, memberlist):
        for m in memberlist:
            if m.uid == uid:
                return m
        return None

    def add_to_in_process(self, t):
        for i in range(len(self.in_process_trans)):
            if self.in_process_trans[i] is None:
                self.in_process_trans[i] = t
                break

    def remove_from_in_process(self, t):
        for i in range(len(self.in_process_trans)):
            if self.in_process_trans[i] == t:
                self.in_process_trans[i] = None
                break

    def find_transaction_block(self, t, last_block):
        current = last_block
        while current is not None:
            for tx in current.trarray:
                if tx == t:
                    return current
            current = current.previous
        return None

# Test code
if __name__ == "__main__":
    print("Testing Members class...")
    
    # Mock DSCoin object
    class DSCoin:
        def __init__(self):
            class Blockchain:
                def __init__(self):
                    self.tr_count = 4
                    self.last_block = None
                    self.insert_block_honest = lambda block: setattr(block, 'dgst', '0000mockdigest')
            
            self.b_chain = Blockchain()
            self.pending_transactions = type('', (), {'add_transaction': lambda self, t: None, 'remove_transaction': lambda self: Transaction()})()
            self.memberlist = []
            self.latest_coin_id = "100000"
    
    # Create test members
    alice = Members("101")
    bob = Members("102")
    
    # Set up test environment
    ds = DSCoin()
    ds.memberlist = [alice, bob]
    
    # Give Alice a coin to send
    alice.mycoins.append(Pair("100001", TransactionBlock([])))
    
    print("Alice initiating coin send to Bob...")
    alice.initiate_coinsend("102", ds)
    
    print("Alice mining a block...")
    alice.mine_coin(ds)
    
    print("Test completed successfully!")

Testing Members class...
Alice initiating coin send to Bob...
Alice mining a block...
Test completed successfully!


In [47]:
class Moderator:
    def initialize_dscoin(self, ds_obj, coin_count):
        members_count = len(ds_obj.memberlist)
        coins_per_member = coin_count // members_count
        
        # Create all transactions
        all_transactions = []
        for i in range(coin_count):
            t = Transaction()
            t.coin_id = str(100000 + i)
            t.source = None  # Moderator
            t.destination = ds_obj.memberlist[i % members_count]
            t.coinsrc_block = None
            all_transactions.append(t)
        
        # Create blocks and distribute coins
        tr_count = ds_obj.b_chain.tr_count
        num_blocks = (coin_count + tr_count - 1) // tr_count  # Ceiling division
        
        for i in range(num_blocks):
            start = i * tr_count
            end = min(start + tr_count, coin_count)
            block_transactions = all_transactions[start:end]
            
            # Fill remaining with None if needed
            while len(block_transactions) < tr_count:
                block_transactions.append(None)
            
            new_block = TransactionBlock(block_transactions)
            if isinstance(ds_obj, DSCoinHonest):
                ds_obj.b_chain.insert_block_honest(new_block)
            else:
                ds_obj.b_chain.insert_block_malicious(new_block)
            
            # Add coins to members' mycoins
            for j in range(start, end):
                if all_transactions[j] is not None:
                    all_transactions[j].destination.mycoins.append(
                        Pair(all_transactions[j].coin_id, new_block)
                    )
        
        # Set latest_coin_id
        ds_obj.latest_coin_id = str(100000 + coin_count - 1)

In [48]:
class Pair:
    def __init__(self, first, second):
        self.first = first
        self.second = second
    
    def get_first(self):
        return self.first
    
    def get_second(self):
        return self.second

In [49]:
class TreeNode:
    def __init__(self):
        self.parent = None
        self.left = None
        self.right = None
        self.val = None

In [50]:
# merkle_tree.py

class CRF:
    """Included CRF implementation to avoid external dependencies"""
    def __init__(self, size):
        self.outputsize = size
    
    def fn(self, s):
        """Simple hash function implementation using SHA-256"""
        import hashlib
        return hashlib.sha256(s.encode()).hexdigest()[:self.outputsize]

class TreeNode:
    """Tree node structure for Merkle Tree"""
    def __init__(self):
        self.parent = None
        self.left = None
        self.right = None
        self.val = None

class MerkleTree:
    def __init__(self):
        self.rootnode = None
        self.numdocs = 0
        self.crf = CRF(64)  # Initialize CRF with output size 64

    def node_init(self, node, left, right, parent, val):
        """Initialize a tree node with given parameters"""
        node.left = left
        node.right = right
        node.parent = parent
        node.val = val

    def get_str(self, tr):
        """Generate the string representation of a transaction"""
        if not hasattr(tr, 'coin_id'):
            return ""
            
        val = tr.coin_id
        
        # Handle source
        if not hasattr(tr, 'source') or tr.source is None:
            val = f"{val}#Genesis"
        else:
            val = f"{val}#{getattr(tr.source, 'uid', 'Unknown')}"
        
        # Handle destination
        if not hasattr(tr, 'destination'):
            val = f"{val}#Unknown"
        else:
            val = f"{val}#{getattr(tr.destination, 'uid', 'Unknown')}"
        
        # Handle coinsrc_block
        if not hasattr(tr, 'coinsrc_block') or tr.coinsrc_block is None:
            val = f"{val}#Genesis"
        else:
            val = f"{val}#{getattr(tr.coinsrc_block, 'dgst', 'Unknown')}"
        
        return self.crf.fn(val)

    def build(self, trarray):
        """Build the Merkle tree from transaction array"""
        if not trarray:
            return ""
            
        self.numdocs = len(trarray)
        q = []
        
        # Create leaf nodes
        for tr in trarray:
            if tr is None:  # Skip None transactions
                continue
                
            nd = TreeNode()
            val = self.get_str(tr)
            self.node_init(nd, None, None, None, val)
            q.append(nd)
        
        # Build tree levels
        while len(q) > 1:
            new_level = []
            for i in range(0, len(q), 2):
                left = q[i]
                right = q[i+1] if i+1 < len(q) else None
                
                parent = TreeNode()
                left_val = left.val if left else ""
                right_val = right.val if right else ""
                data = self.crf.fn(f"{left_val}#{right_val}")
                
                self.node_init(parent, left, right, None, data)
                if left:
                    left.parent = parent
                if right:
                    right.parent = parent
                new_level.append(parent)
            q = new_level
        
        if q:
            self.rootnode = q[0]
            return self.rootnode.val
        return ""

# Test Code
if __name__ == "__main__":
    # Mock Transaction class for testing
    class Transaction:
        def __init__(self):
            self.coin_id = None
            self.source = None
            self.destination = None
            self.coinsrc_block = None
    
    # Mock Member class
    class Member:
        def __init__(self, uid):
            self.uid = uid
    
    # Mock TransactionBlock
    class TransactionBlock:
        def __init__(self):
            self.dgst = "0000mockblockhash"
    
    print("Testing MerkleTree implementation...")
    
    # Create test transactions
    alice = Member("101")
    bob = Member("102")
    genesis_block = TransactionBlock()
    
    tx1 = Transaction()
    tx1.coin_id = "100001"
    tx1.source = None  # Genesis transaction
    tx1.destination = alice
    tx1.coinsrc_block = None
    
    tx2 = Transaction()
    tx2.coin_id = "100002"
    tx2.source = alice
    tx2.destination = bob
    tx2.coinsrc_block = genesis_block
    
    # Build Merkle Tree
    mt = MerkleTree()
    transactions = [tx1, tx2]
    root_hash = mt.build(transactions)
    
    print(f"Merkle root hash: {root_hash}")
    print(f"Number of documents: {mt.numdocs}")
    print("Test completed successfully!")

Testing MerkleTree implementation...
Merkle root hash: f0e58b874aeae3307a27ce17faa28a89c1dbdcae45e501a7070b96172e625a5f
Number of documents: 2
Test completed successfully!


In [51]:
# crf.py

class sha256:
    """SHA-256 implementation included directly in the file"""
    @staticmethod
    def encrypt(password):
        """Compute SHA-256 hash of a string"""
        import hashlib
        return hashlib.sha256(password.encode('utf-8')).hexdigest()

class CRF(sha256):
    """Cryptographic Random Function (CRF) implementation"""
    def __init__(self, size):
        """
        Initialize CRF with specified output size
        
        Args:
            size (int): Number of characters in output hash (max 64)
        """
        self.outputsize = size
        assert self.outputsize <= 64, "Output size cannot exceed 64 characters"

    def fn(self, s):
        """
        Compute the CRF hash of a string
        
        Args:
            s (str): Input string to hash
            
        Returns:
            str: First outputsize characters of SHA-256 hash
        """
        shasum = self.encrypt(s)
        return shasum[:self.outputsize]

# Test code
if __name__ == "__main__":
    print("Testing CRF implementation...")
    
    # Create CRF instance with 64-character output
    crf = CRF(64)
    
    # Test hash function
    test_string = "DSCoin#test#123"
    hashed = crf.fn(test_string)
    
    print(f"Input string: {test_string}")
    print(f"CRF hash (first {crf.outputsize} chars): {hashed}")
    print(f"Hash length: {len(hashed)}")
    print("Test completed successfully!")

Testing CRF implementation...
Input string: DSCoin#test#123
CRF hash (first 64 chars): 1a47b0fd57d6c765aa58e3175976414d96517b20e1901ddb6094392189c5a995
Hash length: 64
Test completed successfully!


In [52]:
import hashlib

class sha256:
    @staticmethod
    def encrypt(password):
        return hashlib.sha256(password.encode('utf-8')).hexdigest()

In [53]:
class Conversion:
    @staticmethod
    def byte_to_hex(hash_bytes):
        return hash_bytes.hex().upper()
    
    @staticmethod
    def hex_to_byte(s):
        return bytes.fromhex(s)

In [54]:
class EmptyQueueException(Exception):
    def __init__(self):
        super().__init__("Queue is empty!")

class MissingTransactionException(Exception):
    def __init__(self):
        super().__init__("Required transaction missing from block chain!")

In [55]:
class Transaction:
    def __init__(self):
        self.coin_id = None
        self.source = None
        self.destination = None
        self.coinsrc_block = None
        self.next = None  # For queue implementation

In [56]:
# dscoin_honest.py

class TransactionQueue:
    """Self-contained TransactionQueue implementation"""
    def __init__(self):
        self.first_transaction = None
        self.last_transaction = None
        self.num_transactions = 0

    def add_transaction(self, transaction):
        if self.first_transaction is None:
            self.first_transaction = transaction
        else:
            self.last_transaction.next = transaction
        self.last_transaction = transaction
        self.num_transactions += 1

    def remove_transaction(self):
        if self.first_transaction is None:
            raise Exception("Queue is empty!")
        removed = self.first_transaction
        self.first_transaction = self.first_transaction.next
        if self.first_transaction is None:
            self.last_transaction = None
        self.num_transactions -= 1
        return removed

    def size(self):
        return self.num_transactions

class CRF:
    """Included CRF implementation"""
    def __init__(self, size):
        self.outputsize = size
    
    def fn(self, s):
        import hashlib
        return hashlib.sha256(s.encode()).hexdigest()[:self.outputsize]

class BlockChainHonest:
    """Self-contained BlockChainHonest implementation"""
    def __init__(self):
        self.tr_count = 4
        self.start_string = "DSCoin"
        self.last_block = None

    def insert_block_honest(self, new_block):
        crf = CRF(64)
        prefix = "0000"
        nonce = 1000000001
        
        prev_digest = self.start_string if self.last_block is None else self.last_block.dgst
        
        while True:
            digest = crf.fn(f"{prev_digest}#{new_block.trsummary}#{nonce}")
            if digest.startswith(prefix):
                new_block.nonce = str(nonce)
                new_block.dgst = digest
                break
            nonce += 1
        
        new_block.previous = self.last_block
        self.last_block = new_block

class DSCoinHonest:
    """Main DSCoinHonest implementation"""
    def __init__(self):
        self.pending_transactions = TransactionQueue()
        self.b_chain = BlockChainHonest()
        self.memberlist = []
        self.latest_coin_id = "99999"  # Will be incremented

# Test Code
if __name__ == "__main__":
    print("Testing DSCoinHonest implementation...")
    
    # Create instance
    ds = DSCoinHonest()
    
    # Verify initialization
    print(f"Initial latest_coin_id: {ds.latest_coin_id}")
    print(f"Pending transactions size: {ds.pending_transactions.size()}")
    print(f"Blockchain last block: {ds.b_chain.last_block}")
    
    print("Test completed successfully!")

Testing DSCoinHonest implementation...
Initial latest_coin_id: 99999
Pending transactions size: 0
Blockchain last block: None
Test completed successfully!


In [57]:
# dscoin_malicious.py

class TransactionQueue:
    """Self-contained TransactionQueue implementation"""
    def __init__(self):
        self.first_transaction = None
        self.last_transaction = None
        self.num_transactions = 0

    def add_transaction(self, transaction):
        if self.first_transaction is None:
            self.first_transaction = transaction
        else:
            self.last_transaction.next = transaction
        self.last_transaction = transaction
        self.num_transactions += 1

    def remove_transaction(self):
        if self.first_transaction is None:
            raise Exception("Queue is empty!")
        removed = self.first_transaction
        self.first_transaction = self.first_transaction.next
        if self.first_transaction is None:
            self.last_transaction = None
        self.num_transactions -= 1
        return removed

    def size(self):
        return self.num_transactions

class CRF:
    """Included CRF implementation"""
    def __init__(self, size):
        self.outputsize = size
    
    def fn(self, s):
        import hashlib
        return hashlib.sha256(s.encode()).hexdigest()[:self.outputsize]

class BlockChainMalicious:
    """Self-contained BlockChainMalicious implementation"""
    def __init__(self):
        self.tr_count = 4
        self.start_string = "DSCoin"
        self.last_blocks_list = [None] * 100  # Max 100 last blocks

    def check_transaction_block(self, tb):
        crf = CRF(64)
        prev_digest = self.start_string if tb.previous is None else tb.previous.dgst
        computed_digest = crf.fn(f"{prev_digest}#{tb.trsummary}#{tb.nonce}")
        
        if computed_digest != tb.dgst:
            return False
        if not tb.dgst.startswith("0000"):
            return False
        
        # Simplified validation - in practice would check Merkle tree and transactions
        return True

    def find_longest_valid_chain(self):
        longest_chain_end = None
        max_length = 0
        
        for last_block in self.last_blocks_list:
            if last_block is None:
                continue
            
            current = last_block
            current_length = 0
            valid_end = None
            
            while current is not None:
                if self.check_transaction_block(current):
                    current_length += 1
                    valid_end = current
                    current = current.previous
                else:
                    if current_length > max_length:
                        max_length = current_length
                        longest_chain_end = valid_end
                    current_length = 0
                    current = current.previous
            
            if current_length > max_length:
                max_length = current_length
                longest_chain_end = valid_end
        
        return longest_chain_end

    def insert_block_malicious(self, new_block):
        last_valid = self.find_longest_valid_chain()
        
        crf = CRF(64)
        prefix = "0000"
        nonce = 1000000001
        
        prev_digest = self.start_string if last_valid is None else last_valid.dgst
        
        while True:
            digest = crf.fn(f"{prev_digest}#{new_block.trsummary}#{nonce}")
            if digest.startswith(prefix):
                new_block.nonce = str(nonce)
                new_block.dgst = digest
                break
            nonce += 1
        
        new_block.previous = last_valid
        
        # Update last_blocks_list
        for i in range(len(self.last_blocks_list)):
            if self.last_blocks_list[i] is None:
                self.last_blocks_list[i] = new_block
                break

class DSCoinMalicious:
    """Main DSCoinMalicious implementation"""
    def __init__(self):
        self.pending_transactions = TransactionQueue()
        self.b_chain = BlockChainMalicious()
        self.memberlist = []
        self.latest_coin_id = "99999"  # Will be incremented

# Test Code
if __name__ == "__main__":
    print("Testing DSCoinMalicious implementation...")
    
    # Create instance
    ds = DSCoinMalicious()
    
    # Verify initialization
    print(f"Initial latest_coin_id: {ds.latest_coin_id}")
    print(f"Pending transactions size: {ds.pending_transactions.size()}")
    print(f"Blockchain last blocks: {len([x for x in ds.b_chain.last_blocks_list if x is not None])}")
    
    print("Test completed successfully!")

Testing DSCoinMalicious implementation...
Initial latest_coin_id: 99999
Pending transactions size: 0
Blockchain last blocks: 0
Test completed successfully!


In [58]:
# dscoin_system.py

# --------------------------
# Helper Classes
# --------------------------

class Pair:
    def __init__(self, first, second):
        self.first = first
        self.second = second

class CRF:
    def __init__(self, size):
        self.outputsize = size
    
    def fn(self, s):
        import hashlib
        return hashlib.sha256(s.encode()).hexdigest()[:self.outputsize]

class Transaction:
    def __init__(self):
        self.coin_id = None
        self.source = None
        self.destination = None
        self.coinsrc_block = None
        self.next = None  # For queue implementation

class TransactionQueue:
    def __init__(self):
        self.first_transaction = None
        self.last_transaction = None
        self.num_transactions = 0

    def add_transaction(self, transaction):
        if self.first_transaction is None:
            self.first_transaction = transaction
        else:
            self.last_transaction.next = transaction
        self.last_transaction = transaction
        self.num_transactions += 1

    def remove_transaction(self):
        if self.first_transaction is None:
            raise Exception("Queue is empty!")
        removed = self.first_transaction
        self.first_transaction = self.first_transaction.next
        if self.first_transaction is None:
            self.last_transaction = None
        self.num_transactions -= 1
        return removed

    def size(self):
        return self.num_transactions

class EmptyQueueException(Exception):
    pass

class MissingTransactionException(Exception):
    pass

# --------------------------
# Blockchain Implementations
# --------------------------

class BlockChainHonest:
    def __init__(self):
        self.tr_count = 4
        self.start_string = "DSCoin"
        self.last_block = None

    def insert_block_honest(self, new_block):
        crf = CRF(64)
        prefix = "0000"
        nonce = 1000000001
        
        prev_digest = self.start_string if self.last_block is None else self.last_block.dgst
        
        while True:
            digest = crf.fn(f"{prev_digest}#{new_block.trsummary}#{nonce}")
            if digest.startswith(prefix):
                new_block.nonce = str(nonce)
                new_block.dgst = digest
                break
            nonce += 1
        
        new_block.previous = self.last_block
        self.last_block = new_block

class BlockChainMalicious:
    def __init__(self):
        self.tr_count = 4
        self.start_string = "DSCoin"
        self.last_blocks_list = [None] * 100

    def check_transaction_block(self, tb):
        crf = CRF(64)
        prev_digest = self.start_string if tb.previous is None else tb.previous.dgst
        computed_digest = crf.fn(f"{prev_digest}#{tb.trsummary}#{tb.nonce}")
        
        if computed_digest != tb.dgst:
            return False
        if not tb.dgst.startswith("0000"):
            return False
        return True

    def find_longest_valid_chain(self):
        longest_chain_end = None
        max_length = 0
        
        for last_block in self.last_blocks_list:
            if last_block is None:
                continue
            
            current = last_block
            current_length = 0
            valid_end = None
            
            while current is not None:
                if self.check_transaction_block(current):
                    current_length += 1
                    valid_end = current
                    current = current.previous
                else:
                    if current_length > max_length:
                        max_length = current_length
                        longest_chain_end = valid_end
                    current_length = 0
                    current = current.previous
            
            if current_length > max_length:
                max_length = current_length
                longest_chain_end = valid_end
        
        return longest_chain_end

    def insert_block_malicious(self, new_block):
        last_valid = self.find_longest_valid_chain()
        
        crf = CRF(64)
        prefix = "0000"
        nonce = 1000000001
        
        prev_digest = self.start_string if last_valid is None else last_valid.dgst
        
        while True:
            digest = crf.fn(f"{prev_digest}#{new_block.trsummary}#{nonce}")
            if digest.startswith(prefix):
                new_block.nonce = str(nonce)
                new_block.dgst = digest
                break
            nonce += 1
        
        new_block.previous = last_valid
        
        for i in range(len(self.last_blocks_list)):
            if self.last_blocks_list[i] is None:
                self.last_blocks_list[i] = new_block
                break

# --------------------------
# Core DSCoin Classes
# --------------------------

class Members:
    def __init__(self, uid):
        self.uid = uid
        self.mycoins = []
        self.in_process_trans = [None] * 100

    def initiate_coinsend(self, dest_uid, ds_obj):
        if not self.mycoins:
            return
        
        coin_pair = self.mycoins.pop(0)
        t = Transaction()
        t.coin_id = coin_pair.first
        t.source = self
        t.destination = self.find_member(dest_uid, ds_obj.memberlist)
        t.coinsrc_block = coin_pair.second
        
        self.add_to_in_process(t)
        ds_obj.pending_transactions.add_transaction(t)

    def finalize_coinsend(self, tobj, ds_obj):
        t_block = self.find_transaction_block(tobj, ds_obj.b_chain.last_block)
        if t_block is None:
            raise MissingTransactionException()
        
        path = [Pair("mock_path1", "mock_path2")]
        proofs = [Pair("mock_proof1", "mock_proof2")]
        
        self.remove_from_in_process(tobj)
        tobj.destination.mycoins.append(Pair(tobj.coin_id, t_block))
        
        return Pair(path, proofs)

    def mine_coin(self, ds_obj):
        transactions = []
        try:
            for _ in range(ds_obj.b_chain.tr_count - 1):
                transactions.append(ds_obj.pending_transactions.remove_transaction())
        except Exception as e:
            if str(e) == "Queue is empty!":
                raise EmptyQueueException()
            raise

        new_coin_id = str(int(ds_obj.latest_coin_id) + 1)
        reward = Transaction()
        reward.coin_id = new_coin_id
        reward.source = None
        reward.destination = self
        reward.coinsrc_block = None
        transactions.append(reward)
        
        new_block = TransactionBlock(transactions)
        if isinstance(ds_obj.b_chain, BlockChainHonest):
            ds_obj.b_chain.insert_block_honest(new_block)
        else:
            ds_obj.b_chain.insert_block_malicious(new_block)
        
        self.mycoins.append(Pair(new_coin_id, new_block))
        ds_obj.latest_coin_id = new_coin_id

    # Helper methods
    def find_member(self, uid, memberlist):
        for m in memberlist:
            if m.uid == uid:
                return m
        return None

    def add_to_in_process(self, t):
        for i in range(len(self.in_process_trans)):
            if self.in_process_trans[i] is None:
                self.in_process_trans[i] = t
                break

    def remove_from_in_process(self, t):
        for i in range(len(self.in_process_trans)):
            if self.in_process_trans[i] == t:
                self.in_process_trans[i] = None
                break

    def find_transaction_block(self, t, last_block):
        current = last_block
        while current is not None:
            for tx in current.trarray:
                if tx == t:
                    return current
            current = current.previous
        return None

class TransactionBlock:
    def __init__(self, transactions):
        self.trarray = transactions
        self.previous = None
        self.dgst = None
        self.trsummary = "mock_trsummary"
        self.nonce = None

class DSCoinHonest:
    def __init__(self):
        self.pending_transactions = TransactionQueue()
        self.b_chain = BlockChainHonest()
        self.memberlist = []
        self.latest_coin_id = "99999"

class DSCoinMalicious:
    def __init__(self):
        self.pending_transactions = TransactionQueue()
        self.b_chain = BlockChainMalicious()
        self.memberlist = []
        self.latest_coin_id = "99999"

class Moderator:
    def initialize_dscoin(self, ds_obj, coin_count):
        members_count = len(ds_obj.memberlist)
        coins_per_member = coin_count // members_count
        
        all_transactions = []
        for i in range(coin_count):
            t = Transaction()
            t.coin_id = str(100000 + i)
            t.source = None
            t.destination = ds_obj.memberlist[i % members_count]
            t.coinsrc_block = None
            all_transactions.append(t)
        
        tr_count = ds_obj.b_chain.tr_count
        num_blocks = (coin_count + tr_count - 1) // tr_count
        
        for i in range(num_blocks):
            start = i * tr_count
            end = min(start + tr_count, coin_count)
            block_transactions = all_transactions[start:end]
            
            while len(block_transactions) < tr_count:
                block_transactions.append(None)
            
            new_block = TransactionBlock(block_transactions)
            if isinstance(ds_obj.b_chain, BlockChainHonest):
                ds_obj.b_chain.insert_block_honest(new_block)
            else:
                ds_obj.b_chain.insert_block_malicious(new_block)
            
            for j in range(start, end):
                if all_transactions[j] is not None:
                    all_transactions[j].destination.mycoins.append(
                        Pair(all_transactions[j].coin_id, new_block)
                    )
        
        ds_obj.latest_coin_id = str(100000 + coin_count - 1)

# --------------------------
# Main Test Code
# --------------------------

def main():
    # Initialize honest DSCoin system
    ds_obj = DSCoinHonest()
    ds_obj.pending_transactions = TransactionQueue()
    ds_obj.b_chain = BlockChainHonest()
    ds_obj.b_chain.tr_count = 4
    
    # Create members
    m1 = Members("101")
    m2 = Members("102")
    m3 = Members("103")
    m4 = Members("104")
    
    # Initialize member lists
    m1.mycoins = []
    m2.mycoins = []
    m3.mycoins = []
    m4.mycoins = []
    
    ds_obj.memberlist = [m1, m2, m3, m4]
    
    # Initialize coins
    mod = Moderator()
    mod.initialize_dscoin(ds_obj, 8)
    
    # Test transactions
    try:
        print("Initializing coin send...")
        m1.initiate_coinsend("102", ds_obj)
        m3.initiate_coinsend("102", ds_obj)
        
        print("Mining block...")
        m2.mine_coin(ds_obj)
        
        print("Finalizing transactions...")
        # Add your finalization tests here
        
        print("All tests completed successfully!")
        
    except (EmptyQueueException, MissingTransactionException) as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

Initializing coin send...
Mining block...
Error: 
